In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
%pip install spotipy
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import time

In [ ]:
grammyAwards = pd.read_csv("data/grammySongs_1999-2019.csv")
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward', 'GrammyYear', 'Genre']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist', 'GrammyYear', 'Genre']).count().reset_index()
grammyAwards = grammyAwards.applymap(lambda s: s.upper() if type(s) == str else s)

# grammyAwards.head(50)

In [ ]:
grammyAwards.head()

In [ ]:
import json
auth_manager = SpotifyClientCredentials(client_id = '7606efb22a6848bdbf48ecd67484b9d5', 
                                        client_secret = '5febce52b67b499db9f7295c5f6839ec')
sp = spotipy.Spotify(auth_manager=auth_manager)
song = sp.search(q=f"track:CITY OF BLINDING LIGHTS, artist:U2", type="track", limit=2)
print(song['tracks']['items'][0]['album']['release_date'])

In [ ]:
for index, row in grammyAwards.iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['GrammyYear']
    try: 
        artistList = artistName.split(",")
        artistName = artistList[0]
        artistName = artistName.split("&")[0]
        artistName = artistName.split(":")[0]
        artistName = artistName.split(";")[0]
        artistName = artistName.split(":")[0]
        artistName = artistName.replace("-", " ")
        if 'FEATURING' in artistName:
            grammyAwards.at[index, 'Artist'] = artistName.split('FEATURING')[0]
            continue
        if 'SOLOIST' in row['Artist']:
            grammyAwards.at[index, 'Artist'] = artistName.split(",")[0]
            continue
        if (artistName != row['Artist']):
            song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
            if len(song['tracks']['items']) == 0:
                song = sp.search(q=f"track:{trackName}", type="track", limit=2)
            artist = song['tracks']['items'][0]['artists'][0]['name'].upper()
    #         artistID = song['tracks']['items'][0]['artists'][0]['uri']
    #         artistGenres = sp.artist(artistID)['genres']
    #         if len(artistGenres) == 0:
    #             print(trackName, artistID)
    #         print(artistGenres)
            grammyAwards.at[index, 'Artist'] = artist
    except:
        print("track not found", trackName)


In [ ]:
grammyAwards.head(50)
grammys_with_genre = grammyAwards

In [ ]:
grammyAwards = grammyAwards[['Name', 'Artist', 'GrammyAward', 'GrammyYear']]
grammyAwards = grammyAwards.groupby(['Name', 'Artist', 'GrammyYear']).count().reset_index()
grammyAwards['GrammyAward']

In [ ]:
grammys_with_genre

In [ ]:
# RUN THIS
grammyAwards['Artist'] = grammyAwards['Artist'].apply(lambda s: s.replace("-", " "))
grammyAwards = grammyAwards.drop_duplicates(subset=['Name', 'Artist'], keep='last').reset_index(drop=True)
# grammyAwards.head(50)

In [ ]:
grammyAwards

In [ ]:
# RUN THIS
# for each song, get the release date, parse to datetime, get the year, set the cell to the year. 
for index, row in grammyAwards.iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['Date']
    try: 
        song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
        if len(song['tracks']['items']) == 0:
            song = sp.search(q=f"track:{trackName}", type="track", limit=2)
        release_date = song['tracks']['items'][0]['album']['release_date']
        release_year = pd.to_datetime(release_date).year
        grammyAwards.at[index, 'Date'] = release_year
    except:
        print("track not found", trackName)
grammyAwards
# song['tracks']['items'][0]['album']['release_date']

In [ ]:
billboardHot100 = pd.read_csv('data/billboardHot100_1999-2019.csv')
billboardHot100 = billboardHot100[['Artists', 'Name', 'Genre', 'Peak.position', 'Weeks.on.chart','Weekly.rank','Date']]
billboardHot100['Weeks.on.chart'] = billboardHot100['Weeks.on.chart'].fillna(1)
billboardHot100['Peak.position'] = billboardHot100['Peak.position'].fillna(100)
billboardHot100['Weekly.rank'] = billboardHot100['Weekly.rank'].fillna(100)
billboardHot100['Genre'] = billboardHot100['Genre'].fillna('None')
billboardHot100['Date'] = billboardHot100['Date'].fillna('1999')
billboardHot100 = billboardHot100.groupby(['Artists', 'Name','Genre']).agg({'Peak.position':'min', 'Weeks.on.chart':'max', 'Weekly.rank':'mean', 'Date':'max'})[['Peak.position','Weeks.on.chart','Weekly.rank', 'Date']].reset_index()
billboardHot100.rename(columns = {'Weekly.rank':'Avg.rank'}, inplace = True)
billboardHot100.rename(
    columns={"Artists":"Artist"}, inplace=True)
billboardHot100['Date'] = pd.to_datetime(billboardHot100['Date'])
billboardHot100['Date'] = billboardHot100['Date'].apply(lambda x : x.year)
billboardHot100 = billboardHot100.applymap(lambda s: s.upper() if type(s) == str else s)
billboardHot100.head()

In [ ]:
for index, row in billboardHot100.iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['Date']
    try: 
        song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
        if len(song['tracks']['items']) == 0:
            song = sp.search(q=f"track:{trackName}", type="track", limit=2)
#         artist = song['tracks']['items'][0]['artists'][0]['name']
#         billboardHot100.at[index, 'Artist'] = artist
        track_id = song['tracks']['items'][0]['uri']
        song_info = sp.audio_features(track_id)           
        billboardHot100.at[index, 'Energy'] = song_info[0].get('energy')
        billboardHot100.at[index, 'Danceability'] = song_info[0].get('danceability')
        billboardHot100.at[index, 'Tempo'] = song_info[0].get('tempo')
        billboardHot100.at[index, 'Valence'] = song_info[0].get('valence')
    except:
        print("track not found", trackName)
    if (index % 500 == 0):
        print(index)
billboardHot100.to_csv("billboard_features.csv")

In [ ]:
# billboardHot100.head(50)
print(grammyAwards[grammyAwards['Artist'] == 'BRUNO MARS'])

In [ ]:
billboardHot100

In [ ]:
print(billboardHot100[billboardHot100['Artist'] == ''])

In [ ]:
grammyAwards

In [ ]:
# grammyAwards[]
grammyAwards.rename(columns={"GrammyYear":"Date"}, inplace=True)
combined_df = grammyAwards.merge(billboardHot100, on=['Name', 'Date'], how='left')
combined_df.head(50)

In [ ]:
%pip install python-levenshtein

In [ ]:
'''
for each one, search in the billboard for the artistName, true if satisfies fuzzy and 
'''
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

for index, row in grammyAwards.iterrows():
    if index % 50 == 0:
        print(index)
    artist1 = row['Artist']
    track = row['Name']
    found = False
    for id2, r2 in billboardHot100.iterrows():
        artist2 = r2['Artist']
        t2 = r2['Name']
        artistRatio = fuzz.partial_ratio(artist1, artist2)
        trackRatio = fuzz.partial_ratio(track, t2)
        if trackRatio > 90 and artistRatio > 90:
            print(artist1, artist2)
            print(track, t2)
            found = True
            grammyAwards.at[index, 'Energy'] = r2['Energy']
            grammyAwards.at[index, 'Danceability'] = r2['Danceability']
            grammyAwards.at[index, 'Tempo'] = r2['Tempo']
            grammyAwards.at[index, 'Valence'] = r2['Valence']
            grammyAwards.at[index, 'Peak.position'] = r2['Peak.position']
            grammyAwards.at[index, 'Weeks.on.chart'] = r2['Weeks.on.chart']
            grammyAwards.at[index, 'Avg.rank'] = r2['Avg.rank']
            break
    if not found:
        grammyAwards.at[index, 'Energy'] = -1
        grammyAwards.at[index, 'Danceability'] = -1
        grammyAwards.at[index, 'Tempo'] = -1
        grammyAwards.at[index, 'Valence'] = -1
        grammyAwards.at[index, 'Peak.position'] = -1
        grammyAwards.at[index, 'Weeks.on.chart'] = -1
        grammyAwards.at[index, 'Avg.rank'] = -1
        print('no match in top 100', track, artist1)
grammyAwards.head(10)
#Peak.position	Weeks.on.chart	Avg.rank

In [ ]:
grammyAwards.to_csv("grammy_award_data.csv")

In [ ]:
bruno_only = billboardHot100[billboardHot100['Artist'] == 'BRUNO MARS']
# print(bruno_only)
for index, row in bruno_only.iterrows():
    artistName = row['Artist']
    trackName = row['Name']
    year = row['Date']
    try: 
        song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
        
        if len(song['tracks']['items']) == 0:
            song = sp.search(q=f"track:{trackName}", type="track", limit=2)
            
#         artist = song['tracks']['items'][0]['artists'][0]['name']
#         bruno_only.at[index, 'Artist'] = artist
        track_id = song['tracks']['items'][0]['uri']
        song_info = sp.audio_features(track_id)           
        bruno_only.at[index, 'Energy'] = song_info[0].get('energy')
        bruno_only.at[index, 'Danceability'] = song_info[0].get('danceability')
        bruno_only.at[index, 'Tempo'] = song_info[0].get('tempo')
        bruno_only.at[index, 'Valence'] = song_info[0].get('valence')
    except:
        print("track not found", trackName)
    if (index % 500 == 0):
        print(index)

In [ ]:
# bruno_only.head(50)

In [ ]:
bruno_grammy = grammyAwards.merge(bruno_only.head(100), on=['Name', 'Artist'], how='left')
bruno_grammy.head(50)

In [ ]:
# kendrick_only = billboardHot100[billboardHot100['Artist'] == 'KENDRICK LAMAR']
# # print(bruno_only)
# for index, row in kendrick_only.iterrows():
#     artistName = row['Artist']
#     trackName = row['Name']
#     year = row['Date']
#     try: 
#         song = sp.search(q=f"track:{trackName}, year:{year}", type="track", limit=2)
        
#         if len(song['tracks']['items']) == 0:
#             song = sp.search(q=f"track:{trackName}", type="track", limit=2)
            
# #         artist = song['tracks']['items'][0]['artists'][0]['name']
# #         bruno_only.at[index, 'Artist'] = artist
#         track_id = song['tracks']['items'][0]['uri']
#         song_info = sp.audio_features(track_id)           
#         kendrick_only.at[index, 'Energy'] = song_info[0].get('energy')
#         kendrick_only.at[index, 'Danceability'] = song_info[0].get('danceability')
#         kendrick_only.at[index, 'Tempo'] = song_info[0].get('tempo')
#         kendrick_only.at[index, 'Valence'] = song_info[0].get('valence')
#     except:
#         print("track not found", trackName)
#     if (index % 500 == 0):
#         print(index)

In [ ]:
kendrick_grammy = grammyAwards.merge(kendrick_only.head(100), on=['Name', 'Artist'], how='left')
# kendrick_grammy.head(50)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

genres_as_list = [g.split(",") for g in billboardHot100['Genre']]
grammy_genres
one_hot = MultiLabelBinarizer()
genre_encoding = one_hot.fit_transform(genres_as_list)
genres = one_hot.classes_

genre_df = pd.DataFrame(genre_encoding, columns = genres)
# genre_df.head()
billboardHot100 = billboardHot100.join(genre_df)
billboardHot100 = billboardHot100.drop(columns=['Genre'])
billboardHot100

In [ ]:
import matplotlib.pyplot as plt
# print(genres)
genre_count = []
genre_dict= {}
for genre in genres:
    count = billboardHot100[genre].sum()
    genre_count.append(count)
    genre_dict[genre] = count
# fig = plt.figure()
print(genre_dict)
plt.bar(genres, genre_count)
plt.show()

In [ ]:
sorted_genres = dict(sorted(genre_dict.items(), key=lambda genre:genre[1]))
print(sorted_genres)

In [ ]:
# list(sorted_genres.keys())
popular_genres_top_50 = list(sorted_genres.keys())[-50:]

In [ ]:
billboardHot100[list(billboardHot100.columns[:6]) + popular_genres_top_50]